In [1]:
import os
import sys
sys.path.append(os.path.abspath(".."))

import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'  # For sharper figures, but it takes more time
from tqdm import tqdm
import scipy as sp
from copy import deepcopy 

from lisatools.utils.constants import *
from lisatools.sensitivity  import AE1SensitivityMatrix
from bbhx.waveformbuild import BBHWaveformFD
import noise_generation as noise_generation
from tools.LISASimulator import LISASimulator
from tools.time_freq_likelihood import TimeFreqLikelihood
from tools.likelihood import get_dh, get_hh
import tools.likelihood as likelihood

from tools.MBHB_differential_evolution import TimeFreqSNR, MBHB_finder, transform_to_01, transform_to_parameters

No CuPy
No CuPy or GPU PhenomHM module.
No CuPy or GPU interpolation available.
No CuPy or GPU response available.


In [2]:
Tobs = YRSID_SI/12
dt = 5.
include_T_channel = False # Set to True if you want to include the T channel in the simulation, otherwise only A and E channels will be included.

wave_gen = BBHWaveformFD(amp_phase_kwargs=dict(run_phenomd=False))
sim = LISASimulator(Tobs=Tobs, dt=dt, wave_gen=wave_gen, include_T_channel=include_T_channel)

m1 = 3e5
m2 = 1.5e5
a1 = 0.2
a2 = 0.4
dist = 5 * PC_SI * 1e9  # distance in Gpc
phi_ref = np.pi/2
f_ref = 0.0
inc = np.pi/3
lam = np.pi/1.
beta = np.pi/4.
psi = np.pi/4.
t_ref = 0.95 * Tobs
#t_ref = round(0.9 * Tobs / dt) * dt  # round to the nearest multiple of dt, to force t_ref to be a part of t_array

parameters = np.array([m1, m2, a1, a2, dist, phi_ref, f_ref, inc, lam, beta, psi, t_ref])

modes = [(2,2), (2,1), (3,3), (3,2), (4,4), (4,3)]
waveform_kwargs = dict(length=1024, direct=False, fill=True, squeeze=False, modes=modes)

data_t, data_f, f_array, t_array, sens_mat = sim(seed = 42, parameters=parameters, waveform_kwargs=waveform_kwargs)
waveform_kwargs.update(freqs=f_array)

print(sim.SNR_optimal()[0])

2876.387525169864


The SNR does not depend on the distance. Change the distance to see that the SNR is the same.
You can calculate the distance based on the amplitude.

In [3]:
guess_distance = dist * 1000
template = wave_gen(
    m1,
    m2,
    a1,
    a2,
    guess_distance, 
    phi_ref,
    f_ref, 
    inc,
    lam,
    beta,
    psi,
    t_ref,
    **waveform_kwargs
)
template = template[0, :2]
print(likelihood.template_snr(data_f, template, AE1SensitivityMatrix(f_array), df=sim.df))

hh = get_hh(template, AE1SensitivityMatrix(f_array), df=sim.df)
dh = get_dh(data_f, template, AE1SensitivityMatrix(f_array), df=sim.df)
amplitude = dh/hh
new_distance = guess_distance /  amplitude

print((new_distance - dist)/(PC_SI*1e9) , (new_distance-dist)*100/dist , "%" )

2876.2995595533453
0.00015291456035319145 0.0030582912070638295 %


Pre-Merger

In [4]:
time_before_merger = 60*60
cutoff_time = t_ref - time_before_merger
max_time = t_ref + 60*60*12

def pre_merger(gravitational_wave_data_t, time_before_merger, t_ref, t_array):
        cutoff_time = t_ref - time_before_merger
        cutoff_index = np.searchsorted(t_array, cutoff_time)
        data_t_truncated = gravitational_wave_data_t[:, :cutoff_index]
        return data_t_truncated, cutoff_index

data_t_truncated, cutoff_index =  pre_merger(data_t, time_before_merger, t_ref, t_array)

The SNR also does not change with the time-frequency SNR with pre-merger. Change guess_distance and see that nothing changes. Also, new_distance remains the same

In [5]:
guess_distance = dist * 100
parameters_new = [
    m1*(1 + 0.001),  # Slightly change m1 to see the effect
    m2,
    a1*(1 + 0.01),  # Slightly change a1 to see the effect
    a2,
    guess_distance,
    phi_ref,
    f_ref,
    inc*(1 + 0.01),
    lam,
    beta,
    psi,
    t_ref
]
analysis = TimeFreqSNR(
    data_t_truncated,
    wave_gen=wave_gen,
    nperseg=5000,
    dt_full=dt,
    cutoff_index=cutoff_index,
    pre_merger=True
)
analysis.get_stft_of_data()
SNR, amplitude = analysis.calculate_time_frequency_SNR(*parameters_new, waveform_kwargs=waveform_kwargs)
new_distance = guess_distance /  amplitude
print((new_distance - dist)/(PC_SI*1e9) , (new_distance-dist)/dist)

-0.022924402780016243 -0.004584880556003249


# Differential Evolution Analysis

In [33]:
boundaries = {}
boundaries['TotalMass'] = [5, 6]
boundaries['MassRatio'] = [0.05, 0.999999]
boundaries['Spin1'] = [-1, 1]
boundaries['Spin2'] = [-1, 1]
boundaries['Distance'] = [1e3, 10e3] # in Mpc
boundaries['Phase'] = [0.0, 2 * np.pi]
boundaries['Inclination'] = [-1, 1]
boundaries['EclipticLongitude'] = [0, 2*np.pi]
boundaries['EclipticLatitude'] = [-1, 1]
boundaries['Polarization'] = [0, np.pi]
boundaries['CoalescenceTime'] = [cutoff_time, max_time]  

parameters_sample = ['TotalMass', 'MassRatio', 'Spin1', 'Spin2', 'Distance', 'Phase', 'Inclination', 'EclipticLongitude', 'EclipticLatitude', 'Polarization', 'CoalescenceTime']


In [76]:
transformed_parameters = [
    m1+m2,  # TotalMass
    m2/m1,  # MassRatio
    a1,      # Spin1
    a2,      # Spin2
    phi_ref,  # Phase
    np.cos(inc),  # Inclination
    lam,     # EclipticLongitude
    np.sin(beta),  # EclipticLatitude
    psi,     # Polarization
    t_ref     # CoalescenceTime
]
params = np.zeros(len(transformed_parameters) + 2)  # +2 for f_ref and distance
params[:4] = transformed_parameters[:4]                                                                                             # mT, q, a1, a2
params[4] = boundaries['Distance'][0] + 0.5 * (boundaries['Distance'][1] - boundaries['Distance'][0])    # dL
params[5] = transformed_parameters[4]                                                                                               # phase                   
params[6] = 0.0                                                                                                         # f_ref is set to 0.0
params[7:] = transformed_parameters[5:]             

params.shape

(12,)

In [55]:
def draw_random_point(boundaries):
    """
    Draw a random point uniformly from each parameter's range.

    Parameters:
        boundaries (dict): Dictionary where each key is a parameter name and 
                           each value is a list [min, max].

    Returns:
        dict: Dictionary with a random value for each parameter.
    """
    return {key: np.random.uniform(low, high) for key, (low, high) in boundaries.items()}
random_point = draw_random_point(boundaries)
#print(random_point)
random_array = [random_point[param] for param in parameters_sample]
print(random_array)

[5.163066015413886, 0.5212651786892647, -0.5101380253867513, -0.5696347165725932, 2558.928891170206, 2.2269076611067757, -0.05029180373328179, 5.190064922032512, 0.04078993688552557, 2.182130917138483, 2503625.6844858043]
